1_ Entender datos (pdf, href formula, href, Unnamed: 22, Unnamed: 23???)
2_ Comprobar columnas o filas duplicadas.
3_ Comprobar columnas o filas NaN.
4_ Eliminar columnas (pdf, href formula, href, Unnamed: 22, Unnamed: 23); lo que no dé valor.
5_ Cambiar nombres columnas.

In [46]:
# Libraries
import numpy as np
import pandas as pd
import re

Cargamos datos de formato CSV

In [2]:
attacks = pd.read_csv("../data/attacks.csv", encoding = "ISO-8859-1")

Visualizamos las primeras filas

In [3]:
attacks.head(3)

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,...,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,NaN,NaN
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,NaN,NaN


 Visualizo las últimas filas hola

In [4]:
attacks.tail(3)

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
25720,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25721,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25722,xx,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Visualizo muestra aleatoria

In [5]:
attacks.sample(3)

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
3071,1980.03.15,15-Mar-1980,1980.0,Sea Disaster,PHILIPPINES,NaN,250 miles southwest of Manila,Sinking of the ferryboat Bongbong 1,2 women,F,...,NaN,"Valley Independent, 3/17/1980",1980.03.15-Bongbong1.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1980.03.15,1980.03.15,3232.0,NaN,NaN
559,2014.05.06,06-May-2014,2014.0,Unprovoked,USA,South Carolina,"Coligny Beach, Hilton Head, Beaufort County",Swimming,Kimberly Popp,F,...,4' to 5' shark,"The Island Packet, 5/7/2014",2014.05.06-Popp.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2014.05.06,2014.05.06,5744.0,NaN,NaN
12517,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Decide a HYPOTHESIS (or hypotheses) with which you will clean the data

..............

ANALISIS EXPLORATORIO DEL DATAFRAME

In [6]:
attacks.shape

(25723, 24)

In [7]:
attacks.info

<bound method DataFrame.info of       Case Number         Date    Year        Type    Country  \
0      2018.06.25  25-Jun-2018  2018.0     Boating        USA   
1      2018.06.18  18-Jun-2018  2018.0  Unprovoked        USA   
2      2018.06.09  09-Jun-2018  2018.0     Invalid        USA   
3      2018.06.08  08-Jun-2018  2018.0  Unprovoked  AUSTRALIA   
4      2018.06.04  04-Jun-2018  2018.0    Provoked     MEXICO   
...           ...          ...     ...         ...        ...   
25718         NaN          NaN     NaN         NaN        NaN   
25719         NaN          NaN     NaN         NaN        NaN   
25720         NaN          NaN     NaN         NaN        NaN   
25721         NaN          NaN     NaN         NaN        NaN   
25722          xx          NaN     NaN         NaN        NaN   

                  Area                        Location     Activity  \
0           California     Oceanside, San Diego County     Paddling   
1              Georgia  St. Simon Island, Gly

In [8]:
attacks.isnull().sum()

Case Number               17021
Date                      19421
Year                      19423
Type                      19425
Country                   19471
Area                      19876
Location                  19961
Activity                  19965
Name                      19631
Sex                       19986
Age                       22252
Injury                    19449
Fatal (Y/N)               19960
Time                      22775
Species                   22259
Investigator or Source    19438
pdf                       19421
href formula              19422
href                      19421
Case Number.1             19421
Case Number.2             19421
original order            19414
Unnamed: 22               25722
Unnamed: 23               25721
dtype: int64

In [9]:
attacks.describe()

,Year,original order
count,6300.000000,6309.000000
mean,1927.272381,3155.999683
std,281.116308,1821.396206
min,0.000000,2.000000
25%,1942.000000,1579.000000
50%,1977.000000,3156.000000
75%,2005.000000,4733.000000
max,2018.000000,6310.000000


In [10]:
attacks.describe().T

,count,mean,std,min,25%,50%,75%,max
Year,6300.0,1927.272381,281.116308,0.0,1942.0,1977.0,2005.0,2018.0
original order,6309.0,3155.999683,1821.396206,2.0,1579.0,3156.0,4733.0,6310.0


COMPRUEBA DATOS NUMËRICOS Y CATEGÓRICOS

In [11]:
# DATOS NUMÉRICOS
numericas = attacks._get_numeric_data()
numericas.head(3)

,Year,original order
0,2018.0,6303.0
1,2018.0,6302.0
2,2018.0,6301.0


In [12]:
# DATOS CATEGÓRICOS
attacks.select_dtypes(include="object").head()

,Case Number,Date,Type,Country,Area,Location,Activity,Name,Sex,Age,...,Time,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,Unnamed: 22,Unnamed: 23
0,2018.06.25,25-Jun-2018,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,...,18h00,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,NaN,NaN
1,2018.06.18,18-Jun-2018,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,11,...,14h00 -15h00,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,NaN,NaN
2,2018.06.09,09-Jun-2018,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,...,07h45,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,NaN,NaN
3,2018.06.08,08-Jun-2018,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,...,NaN,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,NaN,NaN
4,2018.06.04,04-Jun-2018,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,...,NaN,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,NaN,NaN


RENOMBRO COLUMNAS

In [13]:
attacks_copia = attacks.copy()

In [14]:
columnas = list(attacks.columns)
columnas

['Case Number',
 'Date',
 'Year',
 'Type',
 'Country',
 'Area',
 'Location',
 'Activity',
 'Name',
 'Sex ',
 'Age',
 'Injury',
 'Fatal (Y/N)',
 'Time',
 'Species ',
 'Investigator or Source',
 'pdf',
 'href formula',
 'href',
 'Case Number.1',
 'Case Number.2',
 'original order',
 'Unnamed: 22',
 'Unnamed: 23']

In [15]:
diccio_nuevas = {columna: columna.replace(" ", "_").lower() for columna in columnas}

In [16]:
diccio_nuevas

{'Case Number': 'case_number',
 'Date': 'date',
 'Year': 'year',
 'Type': 'type',
 'Country': 'country',
 'Area': 'area',
 'Location': 'location',
 'Activity': 'activity',
 'Name': 'name',
 'Sex ': 'sex_',
 'Age': 'age',
 'Injury': 'injury',
 'Fatal (Y/N)': 'fatal_(y/n)',
 'Time': 'time',
 'Species ': 'species_',
 'Investigator or Source': 'investigator_or_source',
 'pdf': 'pdf',
 'href formula': 'href_formula',
 'href': 'href',
 'Case Number.1': 'case_number.1',
 'Case Number.2': 'case_number.2',
 'original order': 'original_order',
 'Unnamed: 22': 'unnamed:_22',
 'Unnamed: 23': 'unnamed:_23'}

In [17]:
attacks_renombrado = attacks.rename(columns=diccio_nuevas)
attacks_renombrado.sample()

,case_number,date,year,type,country,area,location,activity,name,sex_,...,species_,investigator_or_source,pdf,href_formula,href,case_number.1,case_number.2,original_order,unnamed:_22,unnamed:_23
1880,2002.02.23,23-Feb-2002,2002.0,Unprovoked,TURKS & CAICOS,NaN,NaN,Capsized fishing boat,John Sutton,M,...,NaN,sharkattacks.com,2002.02.23-Sutton.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2002.02.23,2002.02.23,4423.0,NaN,NaN


In [18]:
attacks.rename(columns=diccio_nuevas, inplace=True)

In [19]:
attacks.head(2)

,case_number,date,year,type,country,area,location,activity,name,sex_,...,species_,investigator_or_source,pdf,href_formula,href,case_number.1,case_number.2,original_order,unnamed:_22,unnamed:_23
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,...,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,NaN,NaN


In [20]:
attacks

,case_number,date,year,type,country,area,location,activity,name,sex_,...,species_,investigator_or_source,pdf,href_formula,href,case_number.1,case_number.2,original_order,unnamed:_22,unnamed:_23
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,...,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,NaN,NaN
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,NaN,NaN
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,...,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,NaN,NaN
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,...,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25718,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25719,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25720,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25721,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


EXPLORA DATOS POR COLUMNAS (apuntes)

In [21]:
attacks[attacks["unnamed:_23"].notna()]

,case_number,date,year,type,country,area,location,activity,name,sex_,...,species_,investigator_or_source,pdf,href_formula,href,case_number.1,case_number.2,original_order,unnamed:_22,unnamed:_23
4415,1952.03.30,30-Mar-1952,1952.0,Unprovoked,NETHERLANDS ANTILLES,Curacao,NaN,Went to aid of child being menaced by the shark,A.J. Eggink,M,...,"Bull shark, 2.7 m [9'] was captured & dragged ...","J. Randall, p.352 in Sharks & Survival; H.D. B...",1952.03.30-Eggink.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1952.03.30,1952.03.30,1888.0,NaN,Teramo
5840,1878.09.14.R,Reported 14-Sep-1878,1878.0,Provoked,USA,Connecticut,"Branford, New Haven County",Fishing,Captain Pattison,M,...,NaN,"St. Joseph Herald, 9/14/1878",1878.09.14.R-Pattison.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1878.09.14.R,1878.09.14.R,463.0,NaN,change filename


ELIMINO columnas que no me aportan valor.

In [22]:
attacks.drop(["pdf", "href_formula","href","unnamed:_22", "unnamed:_23"],axis=1,inplace=True)

In [23]:
attacks.head()

,case_number,date,year,type,country,area,location,activity,name,sex_,age,injury,fatal_(y/n),time,species_,investigator_or_source,case_number.1,case_number.2,original_order
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,18h00,White shark,"R. Collier, GSAF",2018.06.25,2018.06.25,6303.0
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,11,Minor injury to left thigh,N,14h00 -15h00,NaN,"K.McMurray, TrackingSharks.com",2018.06.18,2018.06.18,6302.0
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,07h45,NaN,"K.McMurray, TrackingSharks.com",2018.06.09,2018.06.09,6301.0
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,Minor injury to lower leg,N,NaN,2 m shark,"B. Myatt, GSAF",2018.06.08,2018.06.08,6300.0
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,NaN,"Tiger shark, 3m",A .Kipper,2018.06.04,2018.06.04,6299.0


Elimino filas con todo NaN

In [24]:
sharks = attacks.dropna(how="all", axis=0)

In [25]:
sharks.shape

(8703, 19)

LIMPIEZA columna; case_number

In [36]:
case_number = list(attacks.case_number.unique())

In [54]:
def quitar_a(x):
    if ".a" in str(x):
        return x.replace(".a", "")
    else:
        return x

In [55]:
sharks["case_number"] = sharks.case_number.apply(quitar_a)

<ipython-input-55-b9e4096ae751>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sharks["case_number"] = sharks.case_number.apply(quitar_a)


In [42]:
def quitar_b(x):
    if ".b" in str(x):
        return x.replace(".b", "")
    else:
        return x

In [43]:
sharks["case_number"] = sharks.case_number.apply(quitar_b)

<ipython-input-43-7a0d2429a33b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sharks["case_number"] = sharks.case_number.apply(quitar_b)


In [62]:
def quitar_c(x):
    if ".c" in str(x):
        return x.replace(".c", "")
    else:
        return x

In [63]:
sharks["case_number"] = sharks.case_number.apply(quitar_c)

<ipython-input-63-560dba157aa4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sharks["case_number"] = sharks.case_number.apply(quitar_c)


In [65]:
def quitar_d(x):
    if ".c" in str(x):
        return x.replace(".d", "")
    else:
        return x

In [66]:
sharks["case_number"] = sharks.case_number.apply(quitar_d)

<ipython-input-66-327631ea2d19>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sharks["case_number"] = sharks.case_number.apply(quitar_d)


In [67]:
def quitar_R(x):
    if ".R" in str(x):
        return x.replace(".R", "")
    else:
        return x

In [68]:
sharks["case_number"] = sharks.case_number.apply(quitar_R)

<ipython-input-68-46223fb203ba>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sharks["case_number"] = sharks.case_number.apply(quitar_R)


In [ ]:
LIMPIEZA columna; date

In [119]:
def date(x):
    
    return x.extract(r"(\w+-\d+)")

In [122]:
sharks['newcol'] = sharks['date'].str.extract(r"(\w+-\d+)")

<ipython-input-122-65ad4d2e00fd>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sharks['newcol'] = sharks['date'].str.extract(r"(\w+-\d+)")


In [123]:
sharks.sample(3)

,case_number,date,year,type,country,area,location,activity,name,sex_,age,injury,fatal_(y/n),time,species_,investigator_or_source,case_number.1,case_number.2,original_order,newcol
1989,2000.11.20,20-Nov-2000,2000.0,Invalid,AUSTRALIA,South Australia,Ceduna,Shipwrecked,Danny Thorpe,M,47,"Missing, thought to have been taken by a shark",NaN,NaN,Shark involvement not confirmed,"T. Peake, GSAF",2000.11.20,2000.11.20,4314.0,Nov-2000
3240,1975.02.10,10-Feb-1975,1975.0,Unprovoked,AUSTRALIA,South Australia,(Point Sinclair) Penong,Swimming underwater from crayfish cage to a fi...,Wade Shipard,M,12,Right leg severed FATAL,Y,18h00,"White shark, 3 m [10']","A. Sharpe, pp.124-125; Sydney Morning Herald,...",1975.02.10,1975.02.10,3063.0,Feb-1975
718,2012.12.02,02-Dec-2012,2012.0,Unprovoked,AUSTRALIA,New South Wales,Green Island,Spearfishing,male,M,31,Minor puncture wounds to knee,N,16h45,Sandtiger shark,"ABC News, 12/2/2012",2012.12.02,2012.12.02,5585.0,Dec-2012


In [155]:
dates = list(sharks.date)
print(dates[0:5])

['Jun-2018', 'Jun-2018', 'Jun-2018', 'Jun-2018', 'Jun-2018']


In [157]:
newcoll= list(sharks.newcol)
print(newcoll[0:5])

['Jun-2018', 'Jun-2018', 'Jun-2018', 'Jun-2018', 'Jun-2018']


In [158]:
sharks.date=newcoll

In [159]:
print(dates[0:5])

['Jun-2018', 'Jun-2018', 'Jun-2018', 'Jun-2018', 'Jun-2018']


In [161]:
sharks.sample()

,case_number,date,year,type,country,area,location,activity,name,sex_,age,injury,fatal_(y/n),time,species_,investigator_or_source,case_number.1,case_number.2,original_order,newcol
3185,1976.05.02,May-1976,1976.0,Boat,SOUTH AFRICA,Western Cape Province,40 km off Cape Point,Competing in a light tackle game fishing,"6 m skiboat, occupants: Terry McManus, Dan Cla...",NaN,NaN,"Shark following hooked fish, rammed & holed boat",N,NaN,"Mako shark, 180-kg [397-lb]","The Argus, 5/3/1976",1976.05.02,1976.05.02,3118.0,May-1976


LIMPIEZA columna; year

In [162]:
sharks.year.unique()

array([2018., 2017.,   nan, 2016., 2015., 2014., 2013., 2012., 2011.,
       2010., 2009., 2008., 2007., 2006., 2005., 2004., 2003., 2002.,
       2001., 2000., 1999., 1998., 1997., 1996., 1995., 1984., 1994.,
       1993., 1992., 1991., 1990., 1989., 1969., 1988., 1987., 1986.,
       1985., 1983., 1982., 1981., 1980., 1979., 1978., 1977., 1976.,
       1975., 1974., 1973., 1972., 1971., 1970., 1968., 1967., 1966.,
       1965., 1964., 1963., 1962., 1961., 1960., 1959., 1958., 1957.,
       1956., 1955., 1954., 1953., 1952., 1951., 1950., 1949., 1948.,
       1848., 1947., 1946., 1945., 1944., 1943., 1942., 1941., 1940.,
       1939., 1938., 1937., 1936., 1935., 1934., 1933., 1932., 1931.,
       1930., 1929., 1928., 1927., 1926., 1925., 1924., 1923., 1922.,
       1921., 1920., 1919., 1918., 1917., 1916., 1915., 1914., 1913.,
       1912., 1911., 1910., 1909., 1908., 1907., 1906., 1905., 1904.,
       1903., 1902., 1901., 1900., 1899., 1898., 1897., 1896., 1895.,
       1894., 1893.,

In [97]:
def year_(x):
    if ".0" in str(x):
        return x.replace(".0", "")
    else:
        return x

In [98]:
sharks["year"] = sharks.year.apply(year_)

AttributeError: 'float' object has no attribute 'replace'

In [86]:
year_ = list(sharks.year.unique())

In [87]:
year_

[2018.0,
 2017.0,
 nan,
 2016.0,
 2015.0,
 2014.0,
 2013.0,
 2012.0,
 2011.0,
 2010.0,
 2009.0,
 2008.0,
 2007.0,
 2006.0,
 2005.0,
 2004.0,
 2003.0,
 2002.0,
 2001.0,
 2000.0,
 1999.0,
 1998.0,
 1997.0,
 1996.0,
 1995.0,
 1984.0,
 1994.0,
 1993.0,
 1992.0,
 1991.0,
 1990.0,
 1989.0,
 1969.0,
 1988.0,
 1987.0,
 1986.0,
 1985.0,
 1983.0,
 1982.0,
 1981.0,
 1980.0,
 1979.0,
 1978.0,
 1977.0,
 1976.0,
 1975.0,
 1974.0,
 1973.0,
 1972.0,
 1971.0,
 1970.0,
 1968.0,
 1967.0,
 1966.0,
 1965.0,
 1964.0,
 1963.0,
 1962.0,
 1961.0,
 1960.0,
 1959.0,
 1958.0,
 1957.0,
 1956.0,
 1955.0,
 1954.0,
 1953.0,
 1952.0,
 1951.0,
 1950.0,
 1949.0,
 1948.0,
 1848.0,
 1947.0,
 1946.0,
 1945.0,
 1944.0,
 1943.0,
 1942.0,
 1941.0,
 1940.0,
 1939.0,
 1938.0,
 1937.0,
 1936.0,
 1935.0,
 1934.0,
 1933.0,
 1932.0,
 1931.0,
 1930.0,
 1929.0,
 1928.0,
 1927.0,
 1926.0,
 1925.0,
 1924.0,
 1923.0,
 1922.0,
 1921.0,
 1920.0,
 1919.0,
 1918.0,
 1917.0,
 1916.0,
 1915.0,
 1914.0,
 1913.0,
 1912.0,
 1911.0,
 1910.0,
 190

OPERA CON COLUMNAS (SELECCIONA, ELIMINA, Valores únicos ...)

Filtramos los datos por el valor de las columnas

In [ ]:
attacks.area.unique()

BORRAR FILAS Y COLUMNAS. Reset_index y set_index

APPLY!!

Df .loc e .iloc
Seleccionar filas y columnas en Pandas mediante posición con iloc
Seleccionar filas y columnas en Pandas en base a etiquetas con loc

Ordenamos un dataframe
Buscamos valores nulos
Borramos los valores nulos
rellenar los datos que faltan
 Borramos los duplicados

 Agregación de datos
 GroupBy

EXPLORE the DATA and write down what you have found
you can use: df.describe(), df["column"], etc.

DRAW at least 2 GRAPHS that are insightful.

Use at least 5 data cleaning techniques inside a file named clean.ipynb
null values, columns drop, duplicated data, string manipulation, apply fn, categorize, regex, etc.

Show data that validates the conclusions based on your hypoteses in a file named analysis.ipynb

Build a compelling story-telling around your findings. Think of your stakeholders and convince them with your conclusions! (Some slides with few text and pretty plots are normally useful)